In [1]:
!wget https://mcauleylab.ucsd.edu/public_datasets/gdrive/googlelocal/review-Delaware_10.json.gz

--2025-08-28 17:10:09--  https://mcauleylab.ucsd.edu/public_datasets/gdrive/googlelocal/review-Delaware_10.json.gz
Resolving mcauleylab.ucsd.edu (mcauleylab.ucsd.edu)... 169.228.63.88
Connecting to mcauleylab.ucsd.edu (mcauleylab.ucsd.edu)|169.228.63.88|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58276682 (56M) [application/gzip]
Saving to: ‘review-Delaware_10.json.gz’

review-Delaware_10. 100%[===================>]  55.58M  3.86MB/s    in 18s     

2025-08-28 17:10:28 (3.09 MB/s) - ‘review-Delaware_10.json.gz’ saved [58276682/58276682]



In [2]:
!wget https://mcauleylab.ucsd.edu/public_datasets/gdrive/googlelocal/meta-Delaware.json.gz

--2025-08-28 17:10:30--  https://mcauleylab.ucsd.edu/public_datasets/gdrive/googlelocal/meta-Delaware.json.gz
Resolving mcauleylab.ucsd.edu (mcauleylab.ucsd.edu)... 169.228.63.88
Connecting to mcauleylab.ucsd.edu (mcauleylab.ucsd.edu)|169.228.63.88|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3267552 (3.1M) [application/gzip]
Saving to: ‘meta-Delaware.json.gz’

meta-Delaware.json. 100%[===================>]   3.12M  1.76MB/s    in 1.8s    

2025-08-28 17:10:32 (1.76 MB/s) - ‘meta-Delaware.json.gz’ saved [3267552/3267552]



In [1]:
import gzip
import json
import pandas as pd
import re
import itertools

def parse(path):
    g = gzip.open(path, "r")
    for l in g:
        yield json.loads(l)

In [4]:
num_reviews_to_load = 100000
# reviews_data = list(itertools.islice(parse('review-Delaware_10.json.gz'), num_reviews_to_load))
reviews_data = list(parse('review-Delaware_10.json.gz'))
print(f"Data size is {len(reviews_data)}")
df_reviews_new = pd.DataFrame.from_records(reviews_data)

metadata_data = list(parse('meta-Delaware.json.gz'))
df_meta_new = pd.DataFrame.from_records(metadata_data)

Data size is 905537


In [8]:
df_reviews_new = df_reviews_new[['gmap_id', 'user_id', 'name', 'time', 'rating', 'text']]
df_reviews_new.rename(columns={'name': 'user_name'}, inplace=True)

df_reviews_new.dropna(subset=['text'], inplace=True)

def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower() 
    text = re.sub(r'[^a-z\s]', '', text) 
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df_reviews_new['text_clean'] = df_reviews_new['text'].apply(clean_text)

df_meta_new = df_meta_new[['gmap_id', 'name', 'address', 'category', 'avg_rating', 'num_of_reviews', 'price']]

df_meta_new.rename(columns={'name': 'place_name'}, inplace=True)

In [10]:
df_combined = pd.merge(df_reviews_new, df_meta_new, on='gmap_id', how='left')

output_filename = 'ucsd_delaware_reviews_combined.csv'
df_combined.to_csv(output_filename, index=False)